**You need a plotly account.**
Create one, create an API key and run the following command to set your username/password in your home directory : 

    import plotly
    plotly.tools.set_credentials_file(username='DemoAccount', api_key='lr1c37zw81')

Or check https://plot.ly/python/getting-started/ 

**You need to create an api.conf file for your Sigfox backend credentials (see .example file)**

In [1]:
import json
import datetime
import sys
import pandas as pd
import requests
import plotly.graph_objs as go

In [2]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [3]:
from requests.auth import HTTPBasicAuth

In [4]:
init_notebook_mode(connected=True)

**Set your variables**

In [5]:
# Read the API Login/password from api.conf file
with open('api.conf') as f:
    try:
        data = json.load(f)
        if 'APILogin' in data:
            APILogin = data['APILogin']
        if 'APIPassword' in data:
            APIPassword = data['APIPassword']
    except Exception as e: print(e)

In [6]:
#Choose from RC1, RC2, RC3a, RC3c, RC4, RC5, RC6
RC = input()

RC3a


In [7]:
deviceID = input()

73A19C


In [25]:
# datetime(year, month, day, hour, minute, second)
fromDate = datetime.datetime(2017,5,20,0,0,0) #1st Jan 2011 at 0:00am

In [26]:
# you can use datetime.datetime.now() or datetime.datetime(year, month, day, hour, minute, second)
toDate = datetime.datetime.now() #1st Jan 2011 at 0:00am

**Get Data, parse it and Graph it**

In [27]:
fromDateTS = int((fromDate - datetime.datetime(1970, 1, 1))/datetime.timedelta(seconds=1))

In [28]:
toDateTS = int((toDate - datetime.datetime(1970, 1, 1))/datetime.timedelta(seconds=1))

In [29]:
minRSSI = -143.0

In [30]:
maxRSSI = 0.0

In [31]:
df = pd.DataFrame()

In [32]:
url="https://backend.sigfox.com/api/devices/"+deviceID+"/messages?since="+str(fromDateTS)+"&before="+str(toDateTS)

In [33]:
while url:
    print("Fetching messages from :"+url)

    # Call the Backend for messages
    try:
        r = requests.get(url, auth=HTTPBasicAuth(APILogin, APIPassword))
        r.raise_for_status()
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        print(e)
        sys.exit(1)
    except requests.exceptions.HTTPError as e:
        print(e)
        sys.exit(1)
    
    # Load the json
    jData = json.loads(r.content)
    #print(jData)
    
    # go through all messages in Data and take the 
    for msg in jData['data']:
        for rep in msg['rinfos']:
            df2 = pd.DataFrame( [[rep['tap'], rep['freq'], rep['rssi'],rep['delay']]], columns=['tap','freq','rssi','delay'])
            df = df.append(df2)
            maxRSSI = min(maxRSSI, float(rep['rssi']))
            minRSSI = max(minRSSI, float(rep['rssi']))
        
    # If the backend indicates that there is more data in Paging 
    if 'next' in jData['paging']:
        url = jData['paging']['next']
    else:
        url = False

Fetching messages from :https://backend.sigfox.com/api/devices/73A19C/messages?since=1495238400&before=1527517605


In [34]:
df.head()

,tap,freq,rssi,delay
0,8F00,923189534,-128.00,4.329
0,8A22,923184075,-132.00,3.273
0,9B2D,923184448,-135.00,4.836
0,8A22,923189177,-133.00,3.563
0,9B2D,923189526,-134.00,4.636


In [35]:
taps = df.loc[ df['tap'].duplicated() == False].tap

In [36]:
dataGraph = []

In [37]:
for i, tap in enumerate(taps):
    tapG = df.loc[ df['tap'] == tap]
    #print(tapG[['freq','rssi']])
    graph = go.Scatter(
        x = tapG['freq'],
        y = tapG['rssi'],
        mode = 'markers',
        name = tap)
    dataGraph.append(graph)

In [38]:
def RCdef(x):
    return {
        'RC1': {'minFreq' : 868034000, 'maxFreq' : 868226000, 'centralFreq' : 868130000},
        'RC2': {'minFreq' : 902104000, 'maxFreq' : 902296000, 'centralFreq' : 902200000},
        'RC3a': {'minFreq' : 923182000, 'maxFreq' : 923218000, 'centralFreq' : 923200000},
        'RC3c': {'minFreq' : 923104000, 'maxFreq' : 923296000, 'centralFreq' : 923200000},
        'RC4': {'minFreq' : 920704000, 'maxFreq' : 920896000, 'centralFreq' : 920800000},
        'RC5': {'minFreq' : 923204000, 'maxFreq' : 923296000, 'centralFreq' : 923250000},
        'RC6': {'minFreq' : 865104000, 'maxFreq' : 865296000, 'centralFreq' : 865200000}
    }.get(x, {'minFreq' : 868034000, 'maxFreq' : 868226000, 'centralFreq' : 868130000}) 

In [39]:
layout = go.Layout({
    'xaxis': dict(range=[RCdef(RC)['minFreq'],RCdef(RC)['maxFreq']]),
    'shapes': [
        # Line Vertical
        {
            'type': 'line',
            'x0': RCdef(RC)['centralFreq'],
            'y0': minRSSI + 5,
            'x1': RCdef(RC)['centralFreq'],
            'y1': maxRSSI- 5,
            'line': {
                'color': 'rgb(55, 128, 191)',
                'width': 1,
            },
        }
    ]
}
)

In [40]:
fig = go.Figure(data=dataGraph, layout=layout)

In [41]:
iplot(fig)